# Toxic Spans Detection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ast
from termcolor import colored
import string

from nltk import pos_tag
from nltk.corpus import treebank
from nltk.tag import HiddenMarkovModelTrainer
from nltk.tokenize import word_tokenize

import pycrfsuite

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

from IPython.display import clear_output
from itertools import chain
from tqdm import trange, tqdm

import string
from utils.processing import color_toxic_words, remove_symbols, completely_toxic, separate_words, get_index_toxic_words, f1
from utils.basic_models import word2features, sent2features, sent2labels, sent2tokens, token_postag_label

sns.set_style('darkgrid')

## Resumen

1. Quitando símbolos tóxicos (", espacios, !, etc) tenemos los siguientes resultados:
  - Hidden Markov Model: 0.321
  - Conditional Random Fields: 0.551
  - LSTM: 0.561 `(EMBEDDING_DIM = 32, HIDDEN_DIM = 32, EPOCHS = 10)`
2. Quitando símbolos tóxicos (", espacios, !, etc) y agregando como completamente tóxicos todos los que no tienen índices (los que viene como [ ]):
  - Hidden Markov Model: 0.299
  - Conditional Random Fields: 0.546
  - LSTM: 0.556 `(EMBEDDING_DIM = 32, HIDDEN_DIM = 32, EPOCHS = 10)`
  
3. Sin quitar símbolos tóxicos (", espacios, !, etc), es decir, dataset original:
  - Hidden Markov Model: 0.367
  - Conditional Random Fields: 0.501
  - LSTM: 0.535 `(EMBEDDING_DIM = 32, HIDDEN_DIM = 32, EPOCHS = 10)`
  
4. Sin quitar símbolos tóxicos (", espacios, !, etc) y agregando como completamente tóxicos todos los que no tienen índices (los que viene como [ ]):
  - Hidden Markov Model: 0.367
  - Conditional Random Fields: 0.501
  - LSTM: 0.535 `(EMBEDDING_DIM = 32, HIDDEN_DIM = 32, EPOCHS = 10)`
  
Para la LSTM lo mejor parece ser la opción 1. Lo mejor hasta ahora es con `(EMBEDDING_DIM = 20 HIDDEN_DIM = 20 EPOCHS = 25)` (0.581).

- [1. Exploración](#1)
  - [1.1 Mal etiquetados](#1.1)
- [2. Preprocesamiento](#2)
- [3. Modelos](#3)
  - [3.1 Hidden Markov Model](#3.1)
  - [3.2 Conditional Random Fields for Sequence Prediction](#3.2)
  - [3.3 LSTM](#3.3)
- [4. Evaluation](#4)

<a name = "1"></a>
# Exploración

In [2]:
train = pd.read_csv('../data/tsd_train.csv', converters={'spans':ast.literal_eval})
test = pd.read_csv('../data/tsd_trial.csv', converters={'spans':ast.literal_eval})

In [3]:
train.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


Veamos las secciones tóxicas de algunas oraciones:

In [4]:
for i,(index,text) in enumerate(train.head(5).values):
    print('{} - '.format(i) + color_toxic_words(index, text) + '\n'*2)

0 - Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm


1 - I am 56 years old, I am not your fucking junior pal. What you are saying makes no sense. I don't know what you are basing this on. The cheap black market crap is still coming up from Mexican nationals, and the good stuff from BC and Humbolt.  The I5 corridor is still just as busy, and they are still making a ton of money. Nothing has really changed except that the legal market has made it easy for anyone of legal age to buy it.  And the legal market is controlled by corporate growers with million dollar facilities, or a co-op of smaller growers. The Federal government "war on drugs" really has no impact on the legal market in Oregon. I don't see any glutton of weed, whatever that means.


2 - Damn, a whole family. Sad indeed.


3 - What a knucklehead. How can anyone not know this would be offensive??


4 - "who do you think should do the killing?"

Anyone and everyone. This is a

<a name = "1.1"></a>
### ¿Mal etiquetados?

En algunas oraciones pone los espacios, comas y otros signos como caracteres tóxicos, hay que tener cuidado y al final concatenarlos si es que hay dos o más palabras tóxicas juntas. En la competencia parecen concatenarlos, pero no son consistententes en otras partes.

In [5]:
very_toxic = [i for i,x in zip(train.index, train['spans']) if len(x)>100][0:5]

for i,(index,text) in zip(very_toxic, train.values[very_toxic]):
    print('{} - '.format(i) + color_toxic_words(index, text) + '\n'*2)

158 - These idiotic studies are upsetting. Where is the information that is surprising. Any person in any walk of like could and would state these (study) results?


159 - [To say that juxtaposing and making the same a massacre and overtly implying that someone had something to do with that massacre is mere "negligence" is absolutely asinine]
You confer what is determined based upon the evidence is "extremely silly or stupid?"  Is that how you see federal courts and judges function?
[Those "journalists" knew one thing had nothing to do with the other]
What are you waiting for?  You better call Palin and tell her you have evidence that proves the "journalists" knew that the two were not connected because that'll prove there was actual malice.
Wait.  Why don't I hear you dialing?
[Your unrelated SCOTUS case is cute]
Let me get this straight.  The judge in this article used exactly the standard but you deem it as "unrelated?"
You sir, are in complete and completely irrational denial.  It 

Y está el problema de los etiquetados como [ ], que pueden ser completamente tóxicos, sarcásticos o simplemente son no tóxicos.

In [6]:
actually_toxic = [i for i in train.index if len(train.loc[i]['spans']) == 0][0:5]

for i,(index,text) in enumerate(train.values[actually_toxic]):
    print(str(i) + ' ' + color_toxic_words(index, text) + '\n'*2)

0 But, but, but, is NOT a defense.  It's not even a good deflection.  In America today we have Nazis waving the Nazi flag at rallies in our cities. In what capacity does anyone think this is ok and who would not see that as a problem?

Enough with the "Well what about blah blah" garbage.


1 Jim

I think Donald is a con man and a dangerous, narcissistic buffoon. But I don't hate Donald.  He's not worth the emotional effort needed to hate. To say I hate him is to admit that I even care about him, which I don't. What I do care about is the damage he's already caused to this country, and the continuing damage he'll cause to this country and the world until he leaves office.


2 Exposing hypocrites like Trump and Pence is therapeutic for you? Good job!


3 The malignant narcissist, mentally deranged sociopath, bigot, rascist, disingenuous blowhard and pathological liar, has now certified himself as a fascist. How much longer before his GOP sycophants and suckees say, "Enough!", and demand 

<a name = "2"></a>
## Preprocesamiento
Primero sería mejor tener las palabras tóxicas que los caracteres, también hay que tener cuidado cuando marcan un espacio o algún signo de puntuación como tóxico, al final sólo hay que concatenar si hay dos palabras tóxicas juntas (por hacer). También hay que pasar todo a minúscula y hacer el análisis en minúscula.

In [7]:
text = 'fuck you idiot!!!'
index = np.arange(len(text)).tolist()

print(color_toxic_words(index, text))
print(color_toxic_words(remove_symbols(index, text), text))

fuck you idiot!!!
fuck you idiot!!!


Quitamos símbolos.

In [8]:
indices_clean = [remove_symbols(index, text) for index,text in 
                 zip(train['spans'], train['text'])]

train['spans_clean'] = indices_clean

Veamos como se ven si lo hacemos así:

In [9]:
very_toxic = [i for i,x in zip(train.index, train['spans']) if len(x)>100][0:3]

for i, (index,text,index_clean) in zip(very_toxic, train.values[very_toxic]):
    print('{} - '.format(i) + color_toxic_words(index, text))
    print('{} - '.format(i) + color_toxic_words(index_clean, text) + '\n'*2)

158 - These idiotic studies are upsetting. Where is the information that is surprising. Any person in any walk of like could and would state these (study) results?
158 - These idiotic studies are upsetting. Where is the information that is surprising. Any person in any walk of like could and would state these (study) results?


159 - [To say that juxtaposing and making the same a massacre and overtly implying that someone had something to do with that massacre is mere "negligence" is absolutely asinine]
You confer what is determined based upon the evidence is "extremely silly or stupid?"  Is that how you see federal courts and judges function?
[Those "journalists" knew one thing had nothing to do with the other]
What are you waiting for?  You better call Palin and tell her you have evidence that proves the "journalists" knew that the two were not connected because that'll prove there was actual malice.
Wait.  Why don't I hear you dialing?
[Your unrelated SCOTUS case is cute]
Let me get

También pasemos a minúscula el texto para el entrenamiento:

In [10]:
train['text'] = train['text'].apply(lambda x:x.lower())

<a name = "3"></a>
## Modelos

En la competencia usarán el F1 score para evaluar como sigue:

<blockquote cite="http://www.worldwildlife.org/who/index.html">
To evaluate the responses of a system participating in the challenge, we employ the $F_{1}$ score, as in [1]. Let system $A_i$ return a set $S^{t}_{A_{i}}$ of character offsets, for parts of the post found to be toxic. Let $G^{t}$ be the character offsets of the ground truth annotations of $t$. We compute the $F_{1}$ score of system $A_i$ with respect to the ground truth $G$ for post $t$ as follows, where $|·|$ denotes set cardinality.
    
$$ F_{1}^{t}(A_{i}, G) = \dfrac{2 \cdot P^{t}(A_{i}, G) \cdot R^{t}(A_{i}, G)}{P^{t}(A_{i}, G) + R^{t}(A_{i}, G)}$$
    
$$ P^{t}(A_{i}, G) = \dfrac{|S^{t}_{A_{i}} \cap S^{t}_{A_{i}}|}{|S^{t}_{A_{i}}|} $$
    
$$ R^{t}(A_{i}, G) = \dfrac{|S^{t}_{A_{i}} \cap S^{t}_{A_{i}}|}{|S^{t}_{G}|} $$    
    
If $S^{t}_{G}$ is empty for some post $t$ (no gold spans are given for $t$), we set $F_{1}^{t}(A_{i}, G)$ if $S^{t}_{A_{i}}$ is also empty, and $F_{1}^{t}(A_{i}, G) = 0$ otherwise. We finally average $F_{1}^{t}(A_{i}, G)$ over all the posts $t$ of an evaluation dataset $T$ to obtain a single score for system $A_{i}$.

</blockquote>

<a name = "3.1"></a>
### Hidden Markov Model

En nuestro caso debemos etiquetar en `toxic` y `non_toxic`.

In [11]:
train_data = []
for index, text in zip(train['spans_clean'], train['text']):
    toxic_words = [text[i[0]:i[-1]+1] for i in separate_words(index) if len(index) > 0]
    # print(toxic_words)
    
    tokens = word_tokenize(text)
    tagged_tokens = []
    
    for token in tokens:
        if token in toxic_words:
            tagged_tokens.append((token, 'toxic'))
            # Removemos en caso de que se repita posteriormente pero esté como 'non_toxic'
            toxic_words.remove(token) 
        else:
            tagged_tokens.append((token, 'non_toxic'))
            
    train_data.append(tagged_tokens)

Entrenamos:

In [12]:
trainer = HiddenMarkovModelTrainer()
tagger_HMM = trainer.train_supervised(train_data)

Veamos que score tenemos en el dataset de `test` y comparemos nuestras predicciones:

In [13]:
scores_HMM = []
tag_to_ix = {"non_toxic": 0, "toxic": 1}

for i, (gold_index, text) in enumerate(test.values):
    tagged_sentence = tagger_HMM.tag(word_tokenize(text.lower()))   
    tagged_sentence = [(x[0], tag_to_ix[x[1]]) for x in tagged_sentence]
    prediction_index = get_index_toxic_words(text.lower(), tagged_sentence)
    
    if i < 5:
        print(colored('Pred: ', color='cyan', attrs=['bold']) + 
          color_toxic_words(prediction_index, text))   
        print(colored('Gold: ', color='yellow', attrs=['bold']) + 
              color_toxic_words(gold_index, text) + '\n'*2)
    
    scores_HMM.append(f1(prediction_index, gold_index))

Pred: Because he's a moron and a bigot. It's not any more complicated than that.
Gold: Because he's a moron and a bigot. It's not any more complicated than that.


Pred: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.
Gold: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.


Pred: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...
Gold: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...


Pred: Trump Claimed that Russia will never invade the Ukraine, when Russia already has - how stupid can people be?
Gold: Trump Claimed that Russia w

No es muy bueno desafortunadamente.

In [14]:
print('HMM score: {:.3f}'.format(np.mean(scores_HMM)))

HMM score: 0.319


<a name = "3.2"></a>
### Conditional Random Fields for Sequence Prediction

In [15]:
train_data_crf = []
for text, toxic_tags in tqdm(zip(train['text'], train_data), total=len(train_data)):
    pos_tags = token_postag_label(text.lower())
    sentence = []
    for x,y in zip(pos_tags, toxic_tags):
        sentence.append(x + (y[1],))
        
    train_data_crf.append(sentence)

100%|██████████| 7939/7939 [00:09<00:00, 839.87it/s]


In [16]:
X_train = [sent2features(s) for s in train_data_crf]
y_train = [sent2labels(s) for s in train_data_crf]

trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

Probablemente sea buena idea hacer una búsqueda de los mejores hiperparámetros, ya que el performance cambia significativamente dependiendo de los hiperparámetros.

In [17]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 0.01,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

# trainer.params()

In [18]:
%time trainer.train('models/toxic_speech.crfsuite')

tagger_crf = pycrfsuite.Tagger()
tagger_crf.open('models/toxic_speech.crfsuite')

CPU times: user 13.8 s, sys: 2.63 ms, total: 13.8 s
Wall time: 13.8 s


In [19]:
test_data_crf =  []
for text in tqdm(test['text']):
    pos_tags = token_postag_label(text.lower())
    sentence = []
    for x in pos_tags:
        sentence.append(x + ('',))
        
    test_data_crf.append(sentence)

100%|██████████| 690/690 [00:00<00:00, 857.18it/s]


In [20]:
scores_CRF = []
tag_to_ix = {"non_toxic": 0, "toxic": 1}

for i, (text_crf, (gold_index,text)) in enumerate(zip(test_data_crf, test.values)):
    words = [x[0] for x in text_crf]
    tags = tagger_crf.tag(sent2features(text_crf))
    
    tagged_sentence = [(word, tag_to_ix[tag]) for word,tag in zip(words,tags)]
    prediction_index = get_index_toxic_words(text.lower(), tagged_sentence)
    
    if i < 5:
        print(colored('Pred: ', color='cyan', attrs=['bold']) + 
          color_toxic_words(prediction_index, text))   
        print(colored('Gold: ', color='yellow', attrs=['bold']) + 
              color_toxic_words(gold_index, text) + '\n'*2)
    
    scores_CRF.append(f1(prediction_index, gold_index))

Pred: Because he's a moron and a bigot. It's not any more complicated than that.
Gold: Because he's a moron and a bigot. It's not any more complicated than that.


Pred: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.
Gold: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.


Pred: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...
Gold: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...


Pred: Trump Claimed that Russia will never invade the Ukraine, when Russia already has - how stupid can people be?
Gold: Trump Claimed that Russia w

Mejora pero sigue siendo bajo.

In [21]:
print('CRF score: {:.3f}'.format(np.mean(scores_CRF)))

CRF score: 0.552


In [22]:
evaluation = pd.read_csv('../Datos/tsd_test.csv')
evaluation.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Datos/tsd_test.csv'

In [ ]:
indices_evaluation = []

for text in evaluation['text']:
    tagged_sentence = tagger_LSTM(text)   
    tagged_sentence = [(x[0], tag_to_ix[x[1]]) for x in tagged_sentence]
    prediction_index = get_index_toxic_words(text.lower(), tagged_sentence)
    indices_evaluation.append(prediction_index)
    print(colored('Pred: ', color='cyan', attrs=['bold']) + 
          color_toxic_words(prediction_index, text)) 

In [ ]:
evaluation['spans'] = indices_evaluation
evaluation = evaluation[['spans', 'text']]
evaluation.head()

Para la evaluación se debe subir un zip con un archivo txt de la siguiente manera (al final subir el archivo `spans-pred.zip` que se produce):

In [ ]:
predictions = evaluation['spans'].tolist()
ids = evaluation.index.tolist()

In [ ]:
with open("spans-pred.txt", "w") as out:
    for uid, text_scores in zip(ids, predictions):
        out.write(f"{str(uid)}\t{str(text_scores)}\n")
        
# Zip the predictions
! zip -r spans-pred.zip ./spans-pred.* 
! rm spans-pred.txt
! mv spans-pred.zip ../spans-pred.zip